# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd


In [2]:
# Your code here; create the connection and cursor
cur = sqlite3.Connection('data.sqlite').cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [3]:
# Your code here; use a subquery. No join is necessary 
equiv = pd.DataFrame(cur.execute("""SELECT customerName, contactLastName, contactFirstName
                                    FROM customers
                                    WHERE customerNumber IN (SELECT customerNumber
                                                            FROM orders
                                                            WHERE orderDate = '2003-01-31');"""))
equiv.columns = [x[0] for x in cur.description]
equiv

,customerName,contactLastName,contactFirstName
0,Euro+ Shopping Channel,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
query = """SELECT SUM(quantityOrdered) as Total_Units_Sold, count(orderNumber) as Num_Orders, productName
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            GROUP BY productName
                                            ORDER BY Total_Units_Sold DESC;"""
cur.execute(query)

totalorders = pd.DataFrame(cur.fetchall())

totalorders.columns = [x[0] for x in cur.description]
totalorders

,Total_Units_Sold,Num_Orders,productName
0,1808,53,1992 Ferrari 360 Spider red
1,1111,28,1937 Lincoln Berline
2,1085,28,American Airlines: MD-11S
3,1076,28,1941 Chevrolet Special Deluxe Cabriolet
4,1074,28,1930 Buick Marquette Phaeton
...,...,...,...
104,855,25,1999 Indy 500 Monte Carlo SS
105,832,25,1911 Ford Town Car
106,824,25,1936 Mercedes Benz 500k Roadster
107,803,25,1970 Chevy Chevelle SS 454


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [11]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
query = """SELECT COUNT(DISTINCT customerNumber) as Total_people, productName
            FROM products
            JOIN orderdetails
            USING(productCode)
            JOIN orders
            USING(orderNumber)
            GROUP BY productName
            ORDER BY Total_people Desc;"""

df = pd.DataFrame(cur.execute(query))
df.columns = [x[0] for x in cur.description]
df

,Total_people,productName
0,40,1992 Ferrari 360 Spider red
1,27,Boeing X-32A JSF
2,27,1972 Alfa Romeo GTA
3,27,1952 Alpine Renault 1300
4,27,1934 Ford V8 Coupe
...,...,...
104,19,1958 Chevy Corvette Limited Edition
105,18,2002 Chevy Corvette
106,18,1969 Chevrolet Camaro Z28
107,18,1952 Citroen-15CV


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [21]:
# Your code here
query = """SELECT DISTINCT e.employeeNumber, e.lastName, e.firstName, o.city, o.officeCode
            FROM employees e
            JOIN offices o
            USING(officeCode)
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            JOIN orders o
            USING(customerNumber)
            JOIN orderdetails od
            USING(orderNumber)
            WHERE productCode in (SELECT productCode
                                    FROM products
                                    JOIN orderdetails
                                    USING(productCode)
                                    JOIN orders
                                    USING(orderNumber)
                                    GROUP BY productCode
                                    HAVING COUNT(DISTINCT customerNumber) < 20);
                                    """

df = pd.DataFrame(cur.execute(query))
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,city,officeCode
0,1370,Hernandez,Gerard,Paris,4
1,1501,Bott,Larry,London,7
2,1337,Bondur,Loui,Paris,4
3,1166,Thompson,Leslie,San Francisco,1
4,1286,Tseng,Foon Yue,NYC,3
5,1612,Marsh,Peter,Sydney,6
6,1611,Fixter,Andy,Sydney,6
7,1401,Castillo,Pamela,Paris,4
8,1621,Nishi,Mami,Tokyo,5
9,1323,Vanauf,George,NYC,3


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [37]:
# Your code here

query = """SELECT employeeNumber, lastName, firstName, COUNT(customerNumber) AS NumCustomers
            FROM employees e
            JOIN customers c
            ON e.employeeNumber = c.salesRepEmployeeNumber
            GROUP BY employeeNumber, lastName, firstName
            HAVING AVG(creditLimit) > 15000;
            """

df = pd.DataFrame(cur.execute(query))
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,NumCustomers
0,1165,Jennings,Leslie,6
1,1166,Thompson,Leslie,6
2,1188,Firrelli,Julie,6
3,1216,Patterson,Steve,6
4,1286,Tseng,Foon Yue,7
5,1323,Vanauf,George,8
6,1337,Bondur,Loui,6
7,1370,Hernandez,Gerard,7
8,1401,Castillo,Pamela,10
9,1501,Bott,Larry,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!